<a href="https://colab.research.google.com/github/LidiMG/Machine_Learning/blob/main/Regressao_Linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a id='section-zero'></a>

# Regressão Linear

**Objetivos**: Aprender o que é Regressão Linear e introduzir intuições sobre o Método do Gradiente e o Método do Gradiente Estocástico assim como os problemas de regressão de aprendizagem de máquina. Apresentar a biblioteca `SciKit-Learn`.

## Definição - Regressão Linear

> Uma regressão linear faz uma predição simplesmente computando uma soma ponderada dos atributos (*features*), mais uma constante chamada viés (*bias*), também chamado de constante (*intercept*).

<img src="https://github.com/bilhalvadiego/ds-course/blob/main/notebooks/images/reg-linear.png?raw=1" alt="reg-linear" style="width: 400px;"/>

$$ \hat{y} = \theta_0 + \theta_1 x_1 + \theta_2 x_2 + \dots + \theta_n x_n$$

$\hat{y}$ - valor previsto

$\theta$ - parâmetro do modelo

$n$ - número de atributos (*features*)

$x_i$ - o valor do *inésimo* atributo (*feature*)

### Exemplo

$$\mathrm{preço~de~residência} = 4500 + 1000\times \mathrm{quartos} + 120 \times \mathrm{m}^2 + 3000 \times \mathrm{banheiros} - 1500 \times \mathrm{distância~do~centro~km}$$

## Métricas de Desempenho de uma Regressão

<img src="https://github.com/bilhalvadiego/ds-course/blob/main/notebooks/images/erro-reg.png?raw=1" alt="erro-reg" style="width: 400px;"/>

### *Mean Absolute Error* (MAE) - Erro Absoluto Médio

O erro médio absoluto, MAE (da sigla em inglês Mean Absolute Error), é calculado a partir da **média dos erros absolutos**, ou seja, utilizamos o módulo de cada erro para evitar a subestimação, isso porque, o valor é menos afetado por pontos especialmente extremos (outliers).

Cada erro, pode ser interpretado como a diferença entre Y e Ŷ e assim, temos:

$$MAE = \frac{1}{m}\Sigma_{i=1}^{m}{|\hat{y}_i - y_i|}$$

Utilizamos essa medida em séries temporais, pois há casos em que o erro negativo pode zerar o positivo ou dar uma ideia de que o modelo é preciso. Mas aqui, medimos apenas a distância do valor real, independente de ser acima ou abaixo.

### *Mean Squared Error* (MSE) - Erro Quadrático Médio

O erro quadrático médio, MSE (da sigla em inglês Mean Squared Error), é comumente usado para verificar a **acurácia de modelos** e dá um maior peso aos maiores erros, já que, ao ser calculado, cada erro é elevado ao quadrado individualmente e, após isso, a média desses erros quadráticos é calculada.

Usando o mesmo conceito de erro utilizado anteriormente, temos a equação abaixo:

$$MSE = \frac{1}{m}\Sigma_{i=1}^{m}{(\hat{y}_i - y_i)^2}$$

Por conta do expoente ao quadrado que o erro assume, essa métrica é bastante sensível a outliers (valores discrepantes) e, caso tenha muitos erros significativos em sua análise, essa métrica poderá ser extrapolada.

<img src="https://github.com/bilhalvadiego/ds-course/blob/main/notebooks/images/gradient-descent.gif?raw=1" alt="gradient-descent-animation" style="width: 60%;"/>

<img src="https://github.com/bilhalvadiego/ds-course/blob/main/notebooks/images/gradient-descent-2.gif?raw=1" alt="gradient-descent-animation" style="width: 60%;"/>

## Exemplo com o dataset [Boston House Prices](https://scikit-learn.org/stable/datasets/toy_dataset.html#boston-house-prices-dataset)


* $N = 506$
* Atributos: 13
    * `CRIM` crime per capita da região
    * `ZN` proporção de terra residencial
    * `INDUS` proporção terra comercial não-varejista
    * `CHAS` *Dummy* se fica as margens do Charles River (1 ou 0)
    * `NOX` concentração de óxido nítrico (partes por 10 milhões)
    * `RM` número de quartos
    * `AGE` idade da residência
    * `DIS` distância dos cinco centros de emprego de Boston
    * `RAD` acessibilidade às rodovias radiais
    * `TAX` valor do IPTU por 10,000 USD
    * `PTRATIO` relação professor-aluno (*pupil-teacher ratio*) da região
    * `B` proporção de afro-descendentes na região
    * `LSTAT` porcentagem de população de baixa-renda
* **Variável resposta**: valor da casa por 10,000 USD

In [ ]:
import pandas as pd

boston = pd.read_csv('https://raw.githubusercontent.com/bilhalvadiego/ds-course/main/notebooks/data/BostonHousing.csv')


In [ ]:
boston.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [ ]:
X = boston.loc[:,'crim':'lstat']
y = boston['medv']

In [ ]:
print(f"Tamanho de X: {X.shape}")
print(f"Tamanho de y: {y.shape}")

Tamanho de X: (506, 13)
Tamanho de y: (506,)


### Quebrando dataset em `train` e `test`

Usar a função do Scikit-Learn [`sklearn.model_selection.train_test_split()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

#### Argumentos:

* matriz a ser dividida - `X` ou `y`
* `test_size` - `float` ou `int` do tamanho do dataset de teste (padrão $0.25$)
* `train_size` - padrão `1 - test_size`
* `random_state` - `int` - seed do gerador de número randômicos (replicabilidade)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.25,
                                                    random_state=123)

In [ ]:
# Exemplo fora do notebook
import numpy as np
SEED = 5252
np.random.seed(SEED)
np.random.rand()

0.9946915175435148

In [ ]:
print(f"Tamanho de X_train: {X_train.shape}")
print(f"Tamanho de X_test: {X_test.shape}")
print(f"Tamanho de y_train: {y_train.shape}")
print(f"Tamanho de y_test: {y_test.shape}")

Tamanho de X_train: (379, 13)
Tamanho de X_test: (127, 13)
Tamanho de y_train: (379,)
Tamanho de y_test: (127,)


### Regressão Linear
Usar o estimador do Scikit-Learn [`sklearn.linear_model.LinearRegression()`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

#### Retorna:
* Objeto `estimator` do Scikit-Learn

In [ ]:
from sklearn.linear_model import LinearRegression

clf = LinearRegression()

### Classe `Estimators`

* `.fit()` - Treina o Modelo
    * `X`
    * `y`
* `.predict()` - Gera predições do modelo
    * `X`
* `.coef_` - Retorna os coeficientes do modelo ($\theta_i$)
* `.intercept_` - Retorna o viés/constante (*bias/intercept*) do modelo ($\theta_0$)

In [ ]:
clf.fit(X_train, y_train)

LinearRegression()

In [ ]:
clf.coef_.tolist()

[-0.09789103522256513,
 0.042778934770088595,
 0.05914935073037473,
 1.2314983223914302,
 -15.490255817247926,
 4.352157244999688,
 -0.0004691367973203202,
 -1.3772064509938087,
 0.2820857493789268,
 -0.01249194454422125,
 -0.9400115034030564,
 0.006622607126909534,
 -0.5485510540708696]

In [ ]:
# Coeficientes do modelo
for feature, coef in zip(boston.columns.tolist(), clf.coef_.tolist()):
    print(f"{feature}: {round(coef, 2)}")

# Constante do modelo
print(f"Constante: {round(clf.intercept_, 2)}")

crim: -0.1
zn: 0.04
indus: 0.06
chas: 1.23
nox: -15.49
rm: 4.35
age: -0.0
dis: -1.38
rad: 0.28
tax: -0.01
ptratio: -0.94
b: 0.01
lstat: -0.55
Constante: 32.48


### Erro do Modelo


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

y_pred = clf.predict(X_test)

print(f"MSE de Teste: {mean_squared_error(y_test, y_pred):1.1f}")
print(f"MAE de Teste: {mean_absolute_error(y_test, y_pred):1.1f}")

MSE de Teste: 24.8
MAE de Teste: 3.4


In [ ]:
from sklearn.metrics import r2_score

r_quadrado = r2_score(y_test, y_pred)

print('O R² é de {0}. Isto é, {1}% dos casos são explicados pelo modelo.'.format(round(r_quadrado,3), round(r_quadrado*100,1)))

O R² é de 0.686. Isto é, 68.6% dos casos são explicados pelo modelo.


## Atividade - Regressão com o dataset [Diabetes](https://scikit-learn.org/stable/datasets/toy_dataset.html#diabetes-dataset)

* $N = 442$
* Atributos: 10
    * `age`
    * `sex`
    * `bmi` Índice de Massa Corpórea (IMC) - *Body Mass Index* (BMI)
    * `bp` pressão arterial média *blood pressure* (bp)
    * `s1` colesterol total
    * `s2` colesterol LDL
    * `s3` colesterol HDL
    * `s4` colesterol VLDL
    * `s5` triglicerides
    * `s6` glicose
* Variável dependente: medida quantitativa de progressão da diabetes

* Rodem o `LinearRegression()` nos dados de treino e mensure o desempenho nos dados de teste.

>Obs: usar `test_size = 0.25` e `random_state = 123`

In [ ]:
from sklearn.datasets import load_diabetes

diabetes = load_diabetes()
X = diabetes['data']
y = diabetes['target']

In [ ]:
print(f"Nomes dos Atributos: {diabetes['feature_names']}")
print(f"Tamanho de X: {X.shape}")
print(f"Tamanho de y: {y.shape}")

Nomes dos Atributos: ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']
Tamanho de X: (442, 10)
Tamanho de y: (442,)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=18)

In [ ]:
print(f"Tamanho de X_train: {X_train.shape}")
print(f"Tamanho de X_test: {X_test.shape}")
print(f"Tamanho de y_train: {y_train.shape}")
print(f"Tamanho de y_test: {y_test.shape}")

Tamanho de X_train: (331, 10)
Tamanho de X_test: (111, 10)
Tamanho de y_train: (331,)
Tamanho de y_test: (111,)


In [ ]:
from sklearn.linear_model import LinearRegression

clf = LinearRegression()

In [ ]:
clf.fit(X_train, y_train)

LinearRegression()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

y_pred = clf.predict(X_test)

print(f"MSE de Teste: {mean_squared_error(y_test, y_pred):1.1f}")
print(f"MAE de Teste: {mean_absolute_error(y_test, y_pred):1.1f}")

MSE de Teste: 3006.1
MAE de Teste: 47.0


In [ ]:
from sklearn.metrics import r2_score

r_quadrado = r2_score(y_test, y_pred)

print('O R² é de {0}. Isto é, {1}% dos casos são explicados pelo modelo.'.format(round(r_quadrado,3), round(r_quadrado*100,1)))

O R² é de 0.506. Isto é, 50.6% dos casos são explicados pelo modelo.
